In [1]:
import math

import numpy as np
def my_print(x):  # to print 2D numpy arrays without brackets (upto 7 decimal places)
    for row in x:
        print(' '.join(map(lambda x: "{:11.7f}".format(x), row)))

import pandas as pd
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.width', 1000)

import sys

# Project 1: **Molecular Geometry Analysis**

This project demonstrates the calculation of the internal coordinates (bond lengths, bond angles, dihedral angles), moments of inertia, and rotational constants of a polyatomic molecule.

### Step 1: **Read the Coordinate Data from Input**

The input to the program is the set of Cartesian coordinates of the atoms (in bohr) and their associated atomic numbers. The first line in the file is the number of atoms (an integer), while the remaining lines contain the Z-values and $x$, $y$, and $z$ coordinates of each atom (one integer followed by three double-precision floating-point numbers).
$$$$
We store this, excluding the first line, in the following format $$\begin{matrix} Z_1 & x_1 & y_1 & z_1 \\ Z_2 & x_2 & y_2 & z_2 \\ Z_3 & x_3 & y_3 & z_3 \\ \vdots & \vdots & \vdots & \vdots \end{matrix}$$

In [2]:
geom1 = np.genfromtxt("acetaldehyde.dat.txt", skip_header=1) # get the values from the data file (skip the 1st line) and store in a 2D array

### Step 2: **Bond Lengths**

Calculate the interatomic distances using the expression: $$R_{ij}=\sqrt{(x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2}$$ where x, y, and z are Cartesian coordinates and i and j denote atomic indices.

In [3]:
def R(geom,i,j): # bond-length between points i-j
  return math.sqrt((geom[i,1]-geom[j,1])**2+(geom[i,2]-geom[j,2])**2+(geom[i,3]-geom[j,3])**2)


print("\n Interatomic distances (bohr) \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    if j<i:
      print(i, j, "{:9.5f}".format(R(geom1,i,j)) )
print("\n",end="")


 Interatomic distances (bohr) 

1 0   2.84511
2 0   4.55395
2 1   2.29803
3 0   4.19912
3 1   2.09811
3 2   3.81330
4 0   2.06517
4 1   4.04342
4 2   4.84040
4 3   5.87463
5 0   2.07407
5 1   4.05133
5 2   5.89151
5 3   4.83836
5 4   3.38971
6 0   2.07407
6 1   4.05133
6 2   5.89151
6 3   4.83836
6 4   3.38971
6 5   3.33994



### Step 3: **Bond Angles**

Calculate all possible bond angles. For example, the angle, $\hspace{4pt} \phi_{ijk} \hspace{4pt}$, between atoms $\hspace{6pt} i-j-k \hspace{6pt}$, where $\hspace{4pt} j \hspace{4pt}$ is the central atom is given by $$\cos\phi_{ijk}=\hat{\mathbf{e}}_{ji}\cdot\hat{\mathbf{e}}_{jk}$$ where the $\hat{\mathbf{e}}_{ij}$ are unit vectors between the atoms, e.g. $$e_{ij}^x=-\frac{x_i-x_j}{R_{ij}}\ ,\quad e_{ij}^y=-\frac{y_i-y_j}{R_{ij}}\ ,\quad e_{ij}^z=-\frac{z_i-z_j}{R_{ij}}$$

In [4]:
def ex(geom,i,j): # the x-component of the unit vector between points i-j
  return -(geom[i,1]-geom[j,1])/R(geom,i,j)
def ey(geom,i,j): # the y-component of the unit vector between points i-j
  return -(geom[i,2]-geom[j,2])/R(geom,i,j)
def ez(geom,i,j): # the z-component of the unit vector between points i-j
  return -(geom[i,3]-geom[j,3])/R(geom,i,j)
def e(geom,i,j): # the unit vector between points i-j
  return np.array([ ex(geom,i,j), ey(geom,i,j), ez(geom,i,j) ])
def dot(v1, v2): # dot product between two vectors v1, v2 (can also use np.dot)
  return v1[0]*v2[0]+v1[1]*v2[1]+v1[2]*v2[2]
def cosphi(geom,i,j,k): # the cosine of bond-angle between points i-j-k
  return dot(e(geom,j,i), e(geom,j,k))
def phi(geom,i,j,k): # the bond-angle between points i-j-k
  return math.acos(cosphi(geom,i,j,k))

print("\n Bond angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      if j<i and k<j and R(geom1,i,j)<4 and R(geom1,j,k)<4:
        print(i, j, k, "{:12.6f}".format(math.degrees(phi(geom1,i,j,k))) )
print("\n",end="")


 Bond angles 

2 1 0   124.268308
3 1 0   115.479341
3 2 1    28.377448
5 4 0    35.109529
6 4 0    35.109529
6 5 0    36.373677
6 5 4    60.484476



### Step 4: **Out-of-Plane Angles**

Calculate all possible out-of-plane angles. For example, the angle $\hspace{4pt} θ_{ijkl} \hspace{4pt}$ for atom $\hspace{4pt} i \hspace{4pt}$ out of the plane containing atoms $\hspace{6pt} j-k-l \hspace{6pt}$ (with $\hspace{4pt} k \hspace{4pt}$ as the central atom, connected to $\hspace{4pt} i \hspace{4pt}$) is given by $$\sin\theta_{ijkl}=\frac{\hat{\mathbf{e}}_{kj}\times\hat{\mathbf{e}}_{kl}}{\sin\phi_{jkl}}\cdot\hat{\mathbf{e}}_{ki}$$

In [5]:
def cross(v1, v2): # cross product between two vectors v1, v2 (can also use np.cross)
  return np.array([ (v1[1]*v2[2]-v1[2]*v2[1]), (v1[2]*v2[0]-v1[0]*v2[2]), (v1[0]*v2[1]-v1[1]*v2[0]) ])
def sintheta(geom,i,j,k,l): # the sine of out-of-plane angle between point i and plane containing points j-k-l(k as central point)
  return dot(cross(e(geom,k,j), e(geom,k,l)), e(geom,k,i))/math.sin(phi(geom,j,k,l))
def theta(geom,i,j,k,l): #  the out-of-plane angle between point i and plane containing points j-k-l(k as central point)
  if sintheta(geom,i,j,k,l) < -1:
    return math.asin(-1)
  elif sintheta(geom,i,j,k,l) > 1:
    return math.asin(1)
  else:
    return math.asin(sintheta(geom,i,j,k,l))


print("\n Out-of-plane angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      for l in range(len(geom1)):
        if i!=j and i!=k and i!=l and j!=k and l<j and k!=l and R(geom1,i,k)<4 and R(geom1,k,j)<4 and R(geom1,k,l)<4:
          print(i, j, k, l, "{:12.6f}".format(math.degrees(theta(geom1,i,j,k,l))) )
print("\n",end="")


 Out-of-plane angles 

0 3 1 2    -0.000000
0 5 6 4    19.850523
0 6 4 5    19.939726
0 6 5 4   -19.850523
1 5 0 4    53.678778
1 6 0 4   -53.678778
1 6 0 5    54.977064
2 3 1 0     0.000000
3 2 1 0    -0.000000
4 5 0 1   -53.651534
4 5 6 0   -29.885677
4 6 0 1    53.651534
4 6 0 5   -54.869992
4 6 5 0    29.885677
5 4 0 1    53.626323
5 4 6 0    31.064344
5 6 0 1   -56.277112
5 6 0 4    56.194621
5 6 4 0   -30.558964
6 4 0 1   -53.626323
6 4 5 0   -31.064344
6 5 0 1    56.277112
6 5 0 4   -56.194621
6 5 4 0    30.558964



### Step 5: **Torsion/Dihedral Angles**

Calculate all possible torsional angles. For example, the torsional angle $\hspace{4pt} \tau_{ijkl} \hspace{4pt}$ for the atom connectivity $\hspace{6pt} i-j-k-l \hspace{6pt}$ is given by $$\cos\tau_{ijkl}=\frac{(\hat{\mathbf{e}}_{ij}\times\hat{\mathbf{e}}_{jk})\cdot(\hat{\mathbf{e}}_{jk}\times\hat{\mathbf{e}}_{kl})}{\sin\phi_{ijk}\sin\phi_{jkl}}$$


In [6]:
def costau(geom,i,j,k,l):
  return dot(cross(e(geom,i,j),e(geom,j,k)),cross(e(geom,j,k),e(geom,k,l)))/(math.sin(phi(geom,i,j,k))*math.sin(phi(geom,j,k,l)))
def tau(geom,i,j,k,l):
  if costau(geom,i,j,k,l)<-1:
    return math.acos(-1)
  elif costau(geom,i,j,k,l)>1:
    return math.acos(1)
  else:
    return math.acos(costau(geom,i,j,k,l))


print("\n Torsional angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      for l in range(len(geom1)):
        if j<i and k<j and l<k and R(geom1,i,j)<4 and R(geom1,j,k)<4 and R(geom1,k,l)<4:
          print(i, j, k, l, "{:12.6f}".format(math.degrees(tau(geom1,i,j,k,l))) )
print("\n",end="")


 Torsional angles 

3 2 1 0   180.000000
6 5 4 0    36.366799



### Step 6: **Center-of-Mass**

Find the center of mass of the molecule $$x_{cm}=\frac{\sum_im_ix_i}{\sum_im_i}\ ,\quad y_{cm}=\frac{\sum_im_iy_i}{\sum_im_i}\ ,\quad z_{cm}=\frac{\sum_im_iz_i}{\sum_im_i}$$
where $m_i$ is the mass of atom $i$ and the summation runs over all atoms in the molecule
$$$$
We should, first, add another columnn for $m$ values, such that $$\begin{matrix} Z_1 & x_1 & y_1 & z_1 & m_1 \\ Z_2 & x_2 & y_2 & z_2 & m_2 \\ Z_3 & x_3 & y_3 & z_3 & m_3 \\ \vdots & \vdots & \vdots & \vdots & \vdots \end{matrix}$$

In [7]:
masses = np.array([0.,1.00782503207,4.00260325415,7.016004548,9.012182201,11.009305406, 12,14.00307400478,15.99491461956,18.998403224,19.99244017542,
22.98976928087,23.985041699,26.981538627,27.97692653246,30.973761629,31.972070999,34.968852682,39.96238312251,38.963706679,39.962590983,44.955911909,
47.947946281,50.943959507,51.940507472,54.938045141,55.934937475,58.933195048,57.935342907,62.929597474,63.929142222,68.925573587,73.921177767,
74.921596478,79.916521271,78.918337087,85.910610729,84.911789737,87.905612124,88.905848295,89.904704416,92.906378058,97.905408169,98.906254747,
101.904349312,102.905504292,105.903485715,106.90509682,113.90335854,114.903878484,119.902194676,120.903815686,129.906224399,126.904472681,131.904153457,
132.905451932,137.905247237,138.906353267,139.905438706,140.907652769,141.907723297,144.912749023,151.919732425,152.921230339,157.924103912,158.925346757,
163.929174751,164.93032207,165.930293061,168.93421325,173.938862089,174.940771819,179.946549953,180.947995763,183.950931188,186.955753109,191.96148069,
192.96292643,194.964791134,196.966568662,201.970643011,204.974427541,207.976652071,208.980398734,208.982430435,210.987496271,222.017577738,222.01755173,
228.031070292,227.027752127,232.038055325,231.03588399,238.050788247,237.048173444,242.058742611,243.06138108,247.07035354,247.07030708,251.079586788,
252.082978512,257.095104724,258.098431319,255.093241131,260.105504,263.112547,255.107398,259.114500,262.122892,263.128558,265.136151,281.162061,272.153615,
283.171792,283.176451,285.183698,287.191186,292.199786,291.206564,293.214670]) #data taken from https://github.com/psi4/psi3/blob/master/include/masses.h

def add_mass_column(geom): # to dd a column to the geom, for the mass of the atoms
  geom = np.c_[geom, np.zeros(len(geom))]  # add an all-zero column to our geom array
  for i in range(len(geom)): 
    geom[i,4] = masses[int(geom[i,0])]
  return geom

geom1=add_mass_column(geom1) # now the geom array has a 4th column, for the mass of the atoms


def center_of_mass(geom):
  cm = np.zeros([3])
  xcm_num=0
  ycm_num=0
  zcm_num=0
  denom=0
  for i in range(len(geom)):
    xcm_num += geom[i,4]*geom[i,1]
    ycm_num += geom[i,4]*geom[i,2]
    zcm_num += geom[i,4]*geom[i,3]
    denom += geom[i,4]
  cm[0] = xcm_num/denom
  cm[1] = ycm_num/denom
  cm[2] = zcm_num/denom
  return cm

cm = center_of_mass(geom1)


print("\n Molecular center of mass  {:12.8f} {:12.8f} {:12.8f} \n".format(cm[0],cm[1],cm[2]) )


 Molecular center of mass    0.64494925   0.00000000   2.31663790 



### Step 7: **Moments of Inertia**

Calculate elements of the moment of inertia tensor. $$I_{xx}=\sum_im_i\Bigl((y_i-y_{cm})^2+(z_i-z_{cm})^2\Bigr)\ ,\quad I_{yy}=\sum_im_i\Bigl((x_i-x_{cm})^2+(z_i-z_{cm})^2\Bigr)\ ,\quad I_{zz}=\sum_im_i\Bigl((x_i-x_{cm})^2+(y_i-y_{cm})^2\Bigr)\ ,\\ I_{xy}=I_{yx}=-\sum_im_i(x_i-x_{cm})(y_i-y_{cm})\ ,\quad I_{xz}=I_{xz}=-\sum_im_i(x_i-x_{cm})(z_i-z_{cm})\ ,\quad I_{yz}=I_{yz}=-\sum_im_i(y_i-y_{cm})(z_i-z_{cm})$$

In [8]:
I = np.zeros([3,3])
for i in range(len(geom1)):
  I[0,0] += geom1[i,4]*((geom1[i,2]-cm[1])**2+(geom1[i,3]-cm[2])**2)             
  I[1,1] += geom1[i,4]*((geom1[i,1]-cm[0])**2+(geom1[i,3]-cm[2])**2)
  I[2,2] += geom1[i,4]*((geom1[i,1]-cm[0])**2+(geom1[i,2]-cm[2])**2)
  I[0,1] += -geom1[i,4]*(geom1[i,1]-cm[0])*(geom1[i,2]-cm[1])
  I[0,2] += -geom1[i,4]*(geom1[i,1]-cm[0])*(geom1[i,3]-cm[2])
  I[1,2] += -geom1[i,4]*(geom1[i,2]-cm[1])*(geom1[i,3]-cm[2])
I[1,0] = I[0,1]
I[2,0] = I[0,2]
I[2,1] = I[1,2]

print("\n Moment of inertia tensor (amu\u2219bohr\u00b2) \n")
for row in I: # to print 2D numpy arrays without brackets
  print(' '.join(map(lambda I: "{:20.12f}".format(I), row)))
print("\n",end="")


 Moment of inertia tensor (amu∙bohr²) 

    156.154091420701       0.000000000000     -52.855583329239
      0.000000000000     199.371126513842       0.000000000000
    -52.855583329239       0.000000000000     290.739929664163



### Step 8: **Principal Moments of Inertia**

Calculate the eigenvalues of the inertia tensor to obtain the principal moments of inertia $\hspace{3pt} I_a \hspace{1pt}, \hspace{3pt} I_b \hspace{1pt}, \hspace{3pt} I_c$ 
$$$$
The principal axes are ordered such that associated inertia moments decrease, that is, $$I_a \leq I_b \leq I_c$$ Some authors, however, define the $a$ axis as the molecular rotation axis of highest order.

In [9]:
I_eigval, I_eigvec = np.linalg.eig(I)
I_eigval.sort() # sort the eigenvalues in ascending order
Ia = I_eigval[0]
Ib = I_eigval[1]
Ic = I_eigval[2]

print("\n Principal moments of inertia (amu bohr\u00b2) \n {:12.6f} {:12.6f} {:12.6f} \n".format(Ia,Ib,Ic))
const=0.529177249*0.529177249
print("\n Principal moments of inertia (amu \u212b\u00b2) \n {:12.6f} {:12.6f} {:12.6f} \n".format(Ia*const,Ib*const,Ic*const))
const=1.6605402e-24*0.529177249e-8*0.529177249e-8
print("\n Principal moments of inertia (g cm\u00b2) \n {:14.6e} {:14.6e} {:14.6e} \n".format(Ia*const,Ib*const,Ic*const))


 Principal moments of inertia (amu bohr²) 
   137.878035   199.371127   309.015987 


 Principal moments of inertia (amu Å²) 
    38.609788    55.829610    86.533302 


 Principal moments of inertia (g cm²) 
   6.411310e-39   9.270731e-39   1.436920e-38 



### Step 9: **Molecular Type**

Based on the relative values of the principal moments, determine the molecular rotor type
* diatomic
* $ I_a \ll I_b = I_c, \hspace{6pt} I_a \approx 0 \implies $ linear polyatomic
* $ I_a = I_b < I_c \implies $ oblate symmetric top
* $ I_a < I_b = I_c \implies $ prolate symmetric top
* $ I_a = I_b = I_c \implies $ spherical top
* $ I_a < I_b < I_c \implies $ asymmetric top




In [10]:
if len(geom1)==2:
  print("\n Molecule is diatomic. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)<1e-4 and Ia<1e-4:
  print("\n Molecule is a linear polyatomic. \n")
elif abs(Ia-Ib)<1e-4 and abs(Ib-Ic)>1e-4:
  print("\n Molecule is an oblate symmetric top. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)<1e-4:
  print("\n Molecule is a prolate symmetric top. \n")
elif abs(Ia-Ib)<1e-4 and abs(Ib-Ic)<1e-4:
  print("\n Molecule is a spherical top. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)>1e-4:
  print("\n Molecule is an asymmetric top. \n")


 Molecule is an asymmetric top. 



### Step 10: **Rotational Constants**

$$A \geq B \geq C$$
$$$$
$$A=\frac{h}{8\pi^2cI_a}\ , \quad B=\frac{h}{8\pi^2cI_b}\ , \quad C=\frac{h}{8\pi^2cI_c}$$

In [11]:
h=6.6260755e-34 # in m2.kg.s-1
const=h/(8*math.pi*math.pi*1.6605402E-27*(5.29177249e-11**2))*1e-6 
print("\n Rotational constants (MHz) \n    A = {:9.3f}   B = {:9.3f}   C = {:9.3f} \n".format(const/Ia, const/Ib, const/Ic) )
c=2.99792458e10 # in cm.s-1
const=h/(8*math.pi*math.pi*c*1.6605402E-27*(5.29177249e-11**2))
print("\n Rotational constants (cm\u207b\u00b9) \n    A = {:6.4f}   B = {:6.4f}   C = {:6.4f} \n".format(const/Ia, const/Ib, const/Ic) )


 Rotational constants (MHz) 
    A = 13089.403   B =  9052.169   C =  5840.284 


 Rotational constants (cm⁻¹) 
    A = 0.4366   B = 0.3019   C = 0.1948 



# Project 2: **Harmonic Vibrational Analysis**


This project demonstrates a normal coordinate/harmonic vibrational frequency calculation.

### Step 1: **Read the Coordinate Data**

The coordinate data are given in a format identical to that for Project 1. The test case for the remainder of this project is the water molecule, optimized at the SCF/DZP level of theory.

In [12]:
geom2=np.genfromtxt("h2o_geom.txt", skip_header=1)

### Step 2: **Read the Cartesian Hessian Data**

The primary input data for the harmonic vibrational calculation is the Hessian matrix, which consists of second derivatives of the energy with respect to atomic positions $$F_{ij}=\frac{\partial^2V}{\partial q_i\partial q_j}$$
$$$$
The first integer in the file is the number of atoms (which you should compare to the corresponding value from the geometry file as a test of consistency), while the remaining values have the following format $$\begin{matrix} F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} \\ F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} \\ F_{x_1,x_3} & F_{x_1,y_3} & F_{x_1,z_3} \\ F_{y_1,x_1} & F_{y_1,y_1} & F_{y_1,z_1} \\ F_{y_1,x_2} & F_{y_1,y_2} & F_{y_1,z_2} \\ \vdots & \vdots & \vdots \\ F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} \end{matrix}$$

While the format of the input file is rectangular, the Hessian should be rearranged in a square matrix $$\begin{matrix}F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} & F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} & F_{x_1,x_3} & F_{x_1,y_3} & F_{x_1,z_3} \\ F_{y_1,x_1} & F_{y_1,y_1} & F_{y_1,z_1} & F_{y_1,x_2} & F_{y_1,y_2} & F_{y_1,z_2} & F_{y_1,x_3} & F_{y_1,y_3} & F_{y_1,z_3} \\ \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \ \ \\ F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} & F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} & F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3}\end{matrix}$$

In [13]:
hess2=np.genfromtxt("h2o_hessian.txt", skip_header=1)

def making_square_hess(hess,geom):
  H = np.zeros([3*len(geom),3*len(geom)])
  for i in range(3*len(geom)):
    for j in range(len(geom)):
      for k in range(len(geom)):
        H[i,(k+j*3)]=hess[(i*3+j),k]
  return H

F = making_square_hess(hess2,geom2)

### Step 3: **Mass-Weight the Hessian Matrix**

Divide each element of the Hessian matrix by the product of square-roots of the masses of the atoms associated with the given coordinates $$F_{ij}^M=\frac{F_{ij}}{\sqrt{m_im_j}}$$ where $m_i$ represents the mass of the atom corresponding to atom $i$

In [14]:
geom2=add_mass_column(geom2)

def making_mass_hess(square_hess,geom):
  mass_arr = np.zeros([3*len(geom)])
  for i in range(len(geom)):
    for j in range(3):
      mass_arr[i*3+j]=geom[i,4]
  mass_hess = np.zeros([3*len(geom),3*len(geom)])
  for i in range(3*len(geom)):
    for j in range(3*len(geom)):
      mass_hess[i,j]=square_hess[i,j]/(math.sqrt(mass_arr[i]*mass_arr[j]))
  return mass_hess

F_M = making_mass_hess(F,geom2)


print("\n For the H\u2082O test case, the 9\u00d79 mass-weighted Hessian is \n")
my_print(F_M)
print("\n",end="")


 For the H₂O test case, the 9×9 mass-weighted Hessian is 

  0.0057996   0.0000000   0.0000000  -0.0115523   0.0000000   0.0000000  -0.0115523   0.0000000   0.0000000
  0.0000000   0.0198271   0.0000000   0.0000000  -0.0394937   0.0199304   0.0000000  -0.0394937  -0.0199304
  0.0000000   0.0000000   0.0175112   0.0000000   0.0086617  -0.0348807   0.0000000  -0.0086617  -0.0348807
 -0.0115523   0.0000000   0.0000000   0.0510672   0.0000000   0.0000000  -0.0050452   0.0000000   0.0000000
  0.0000000  -0.0394937   0.0086617   0.0000000   0.1716643  -0.0569527   0.0000000  -0.0143291   0.0224462
  0.0000000   0.0199304  -0.0348807   0.0000000  -0.0569527   0.1258525   0.0000000  -0.0224462   0.0131055
 -0.0115523   0.0000000   0.0000000  -0.0050452   0.0000000   0.0000000   0.0510672   0.0000000   0.0000000
  0.0000000  -0.0394937  -0.0086617   0.0000000  -0.0143291  -0.0224462   0.0000000   0.1716643   0.0569527
  0.0000000  -0.0199304  -0.0348807   0.0000000   0.0224462   0.0131055   0.

### Step 4: **Eigenvalues of the Mass-Weighted Hessian Matrix**

Compute the eigenvalues of the mass-weighted Hessian.



In [15]:
F_M_eigval, F_M_eigvec = np.linalg.eig(F_M)
F_M_eigval = np.sort(F_M_eigval)[::-1] # sort the eigenvalues in descending order

print("\n The eigenvalues of the mass-weighted Hessian for the H\u2082O test case (in atomic units) are \n")
np.savetxt(sys.stdout, F_M_eigval, fmt="%18.10f")  # this is to print the values vertically, without brackets
print("\n",end="")


 The eigenvalues of the mass-weighted Hessian for the H₂O test case (in atomic units) are 

      0.2351542374
      0.2107113147
      0.1317512795
      0.0561123956
      0.0547551460
      0.0518216600
      0.0000000000
     -0.0000000000
     -0.0000000000



### Step 5: **Compute the Harmonic Vibrational Frequencies**

The vibrational frequencies are proportional to the squareroot of the eigenvalues of the mass-weighted Hessian $$\omega_i=constant\sqrt{λ_i}$$

In [16]:
F_M_eigval = F_M_eigval/5.29177249e-9  # convert from bohr-1 to cm-1 ?

omega = np.sqrt(np.absolute(F_M_eigval)) # I don't know the value of the constant, so i just did the square-root

print("\n The harmonic vibrational frequencies for the H\u2082O test case (in cm\u207b\u00b9) are \n")
np.savetxt(sys.stdout, omega, fmt="%12.4f")  # this is to print the values vertically, without brackets
print("\n",end="")


 The harmonic vibrational frequencies for the H₂O test case (in cm⁻¹) are 

   6666.1614
   6310.2032
   4989.7276
   3256.3331
   3216.7098
   3129.3568
      0.0561
      0.0000
      0.0000



Note that there are only three zero frequencies in this case when there should be six. This is because the structure used in the computation is not a stationary point on the potential energy surface, and thus the three “rotational frequencies” are non-zero.

# Project 3: **The Hartree-Fock self-consistent field (SCF) procedure**


This project is a simple implementation of the self-consistent-field method. 

## **Integral Preprocessing**

In [17]:
molecule = "H\u2082O"

N = 10 #number of total electrons

n_basis = np.array([5,1,1]) # we have 5 AOs on 1st atom (O), 1 AOs on 2nd atom (H), 1 AOs on 3rd atom (H)

### Step 1: **Nuclear Repulsion Energy**

Read the nuclear repulsion energy $E_{nuc}$ 

In [18]:
E_nuc = np.genfromtxt("enuc.dat.txt")

### Step 2: **One-Electron Integrals**

Read the AO-basis overlap $$S_{\mu\nu}=\int \phi_\mu^*(\mathbf{r})\phi_\nu(\mathbf{r})d\mathbf{r}$$ kinetic-energy $$T_{\mu\nu}=\int \phi_\mu^*(\mathbf{r}) \left(-\frac{1}{2}\nabla_r^2 \right) \phi_\nu(\mathbf{r})d\mathbf{r}$$ nuclear-attraction integrals $$V_{\mu\nu}=\int \phi_\mu^*(\mathbf{r}) \left(-\sum_A^N\frac{Z}{r_A} \right) \phi_\nu(\mathbf{r})d\mathbf{r}$$ and store them in appropriately constructed matrices. 
$$$$
Then form the "core Hamiltonian" $$H_{\mu\nu}^{core}=T_{\mu\nu} +V_{\mu\nu}$$
$$$$
Note that the one-electron integrals provided include only the permutationally unique integrals, in the format $$\begin{matrix} 1 & 1 & A_{11} \\ 2 & 1 & A_{21} \\ 2 & 2 & A_{22} \\ 3 & 1 & A_{31} \\ \vdots & \ \ \vdots & \ \ \vdots \ \ \\ 7 & 7 & A_{77} \end{matrix}$$ but we should store the full matrices for convenience. $$\begin{matrix} A_{11} & \color{gray}{A_{21}} & \color{gray}{A_{31}} & \color{gray}{\cdots} & \color{gray}{A_{71}} \\ A_{21} & A_{22} & \color{gray}{A_{32}} & \color{gray}{\cdots} & \color{gray}{A_{72}} \\ A_{31} & A_{32} & A_{33} & \color{gray}{\cdots} & \color{gray}{A_{73}} \\ \vdots & \vdots & \vdots & \ddots & \color{gray}{\vdots} \\ A_{71} & A_{72} & A_{73} & \cdots & A_{77}\end{matrix} % usually I'd have used \textcolor instead of \color $$

Note also that the AO indices on the integrals in the files start with "$\ \ _1$" rather than "$\ \ _0$".
 

In [19]:
Smunu = np.genfromtxt("s.dat.txt")
Tmunu = np.genfromtxt("t.dat.txt")
Vmunu = np.genfromtxt("v.dat.txt")

nAO = int(np.max(Smunu[:,0])) # to get the number of AO, pick-up the max value from 0th column of Smunu
# could've got it from 0th/1th column of Smunu/Tmunu/Vmunu, or even 0th/1th/2th/3th column of eri 


def make_2Dmatrix(X):
  A = np.zeros([nAO,nAO])
  for a in range(len(X)): # it goes over all the lines of the input array
    mu=int(X[a,0]-1)
    nu=int(X[a,1]-1)
    A[mu,nu]=X[a,2]
    A[nu,mu]=X[a,2]
  return A

S = make_2Dmatrix(Smunu)
T = make_2Dmatrix(Tmunu)
V = make_2Dmatrix(Vmunu)

def add_matrices(X,Y):
  Z = np.zeros([len(X),len(X)])
  for i in range(len(X)):
    for j in range(len(X)):
      Z[i,j] = X[i,j]+Y[i,j]
  return Z

H_core = add_matrices(T,V)

print("\n The core Hamiltonian for the ", molecule, " test case is \n")
my_print(H_core)
print("\n",end="")


 The core Hamiltonian for the  H₂O  test case is 

-32.5773954  -7.5788328   0.0000000  -0.0144738   0.0000000  -1.2401023  -1.2401023
 -7.5788328  -9.2009433   0.0000000  -0.1768902   0.0000000  -2.9067098  -2.9067098
  0.0000000   0.0000000  -7.4588193   0.0000000   0.0000000  -1.6751501   1.6751501
 -0.0144738  -0.1768902   0.0000000  -7.4153118   0.0000000  -1.3568683  -1.3568683
  0.0000000   0.0000000   0.0000000   0.0000000  -7.3471449   0.0000000   0.0000000
 -1.2401023  -2.9067098  -1.6751501  -1.3568683   0.0000000  -4.5401711  -1.0711459
 -1.2401023  -2.9067098   1.6751501  -1.3568683   0.0000000  -1.0711459  -4.5401711



### Step 3: **Two-Electron Integrals**

Read the two-electron repulsion integrals $$(\mu\nu|\lambda\sigma) = \int \phi_\mu^*(\mathbf{r}_1)\phi_\nu(\mathbf{r}_1)\phi_\lambda^*(\mathbf{r}_2)\phi_\sigma(\mathbf{r}_2)d\mathbf{r}_1 d\mathbf{r}_2$$ Hence, the integrals obey the eight-fold permutational symmetry relationships $$(\mu\nu|\lambda\sigma) = (\nu\mu|\lambda\sigma) = (\mu\nu|\sigma\lambda) = (\nu\mu|\sigma\lambda) = (\lambda\sigma|\mu\nu) = (\sigma\lambda|\mu\nu) = (\lambda\sigma|\nu\mu) = (\sigma\lambda|\nu\mu)$$
$$$$
Note that only the permutationally unique integrals are provided in the file, in the format (not necessarily in the same order) $$\begin{matrix} 1 & 1 & 1 & 1 & A_{1111}\\ 2 & 1 & 1 & 1 & A_{2111} \\ 2 & 2 & 1 & 1 & A_{2211} \\ 2 & 1 & 2 & 1 & A_{2121} \\ 2 & 2 & 2 & 2 & A_{2222} \\ \vdots & \vdots & \vdots & \vdots & \vdots \end{matrix}$$ but we should store the full 4-dimensional tensor.

Note also that the AO indices on the integrals in the files start with "$\ \ _1$" rather than "$\ \ _0$".

In [20]:
eri = np.genfromtxt("eri.dat.txt")

def make_4Dtensor(X):
  A = np.zeros([nAO,nAO,nAO,nAO])
  for a in range(len(X)): # it goes over all the lines of the input array
    mu=int(X[a,0]-1)
    nu=int(X[a,1]-1)
    lam=int(X[a,2]-1)
    sig=int(X[a,3]-1)
    A[mu,nu,lam,sig]=X[a,4]
    A[nu,mu,lam,sig]=X[a,4]
    A[mu,nu,sig,lam]=X[a,4]
    A[nu,mu,sig,lam]=X[a,4]
    A[lam,sig,mu,nu]=X[a,4]
    A[sig,lam,mu,nu]=X[a,4]
    A[lam,sig,nu,mu]=X[a,4]
    A[sig,lam,nu,mu]=X[a,4]
  return A

Eri = make_4Dtensor(eri)

## **Orthogonalisation of the Basis Set : The $\mathbf{S^{-1/2}}$ Matrix**

### Step 4: **Build the Orthogonalization Matrix**

Diagonalize the overlap matrix $$ \mathbf{S} \hspace{1pt} \mathbf{L_S^{\phantom{}}}= \mathbf{L_S^{\phantom{}}} \hspace{1pt} \mathbf{\Lambda_S^{\phantom{}}} $$ $$\implies \mathbf{\Lambda_S^{\phantom{}}} = \mathbf{L_S^\mathsf{T}} \hspace{3pt} \mathbf{S} \hspace{2pt} \mathbf{L_S^{\phantom{}}} $$ where $\mathbf{L_S^{\phantom{}}}$ is the matrix of eigenvectors (columns) and $\mathbf{\Lambda_S}^{\phantom{}}$ is the diagonal matrix of corresponding eigenvalues.
$$$$
Build the symmetric orthogonalization matrix using $$ \mathbf{S^{-1/2}} = \mathbf{L_S^{\phantom{}}} \hspace{3pt} \mathbf{\Lambda_S^{-1/2}} \hspace{3pt} \mathbf{L_S^\mathsf{T}}$$ where the $\hspace{4pt} \mathbf{^\mathsf{T}} $ denotes the matrix transpose.

In [21]:
S_eigval, S_eigvec = np.linalg.eig(S)

lam_S = np.dot(np.transpose(S_eigvec), np.dot(S, S_eigvec)) #we can also use matrix inverse instead of transpose 

def powerofdiagonalmatrix(X,n): # to ^n only to the diagonal elements of the matrix
  for i in range(len(X)):
    X[i,i] = X[i,i]**n
  return X

S_minhalf = np.dot(S_eigvec, np.dot(powerofdiagonalmatrix(lam_S,(-1/2)), np.transpose(S_eigvec))) #we can also use matrix inverse instead of transpose 

print("\n The S\u207b\u00b9\u2e0d\u00b2 matrix for the ", molecule, " test case is \n")
my_print(S_minhalf)
print("\n",end="")


 The S⁻¹⸍² matrix for the  H₂O  test case is 

  1.0236346  -0.1368547   0.0000000  -0.0074873   0.0000000   0.0190279   0.0190279
 -0.1368547   1.1578632  -0.0000000   0.0721601   0.0000000  -0.2223326  -0.2223326
  0.0000000  -0.0000000   1.0733148  -0.0000000   0.0000000  -0.1757583   0.1757583
 -0.0074873   0.0721601  -0.0000000   1.0383050   0.0000000  -0.1184626  -0.1184626
  0.0000000   0.0000000   0.0000000   0.0000000   1.0000000   0.0000000   0.0000000
  0.0190279  -0.2223326  -0.1757583  -0.1184626   0.0000000   1.1297234  -0.0625975
  0.0190279  -0.2223326   0.1757583  -0.1184626   0.0000000  -0.0625975   1.1297234



## **The Initial (Guess) Density Matrix**

### Step 5: **Build the Initial Guess Density**

Form an initial (guess) Fock matrix in the orthonormal AO basis using the core Hamiltonian as a guess $$\mathbf{F^{\prime}}^0 = \mathbf{S^{-1/2}}^\mathsf{T} \hspace{3pt} \mathbf{H^{core}} \hspace{2pt} \mathbf{S^{-1/2}}$$ 
$$$$
Diagonalize the Fock matrix $$\mathbf{F^{\prime}}^0 \hspace{2pt} \mathbf{C^\prime}^0 = \mathbf{C^\prime}^0 \hspace{2pt} \boldsymbol{\epsilon}^0$$ $$\implies \boldsymbol{\epsilon}^0 = \mathbf{C^{\prime\mathsf{T}}}^0 \hspace{3pt} \mathbf{F^{\prime}}^0 \hspace{2pt} \mathbf{C^\prime}^0$$ Note that the $\boldsymbol{\epsilon}^0$ matrix contains the initial MO energies.
$$$$
Transform the eigenvectors into the original (non-orthogonal) AO basis $$\mathbf{C}^0=\mathbf{S^{-1/2}} \hspace{2pt} \mathbf{C^\prime}^0$$
$$$$
Build the density matrix using the occupied MOs $$D_{\mu\nu}^0 = \sum_a^{occ} \hspace{2pt} C^0_{\mu a} \hspace{1pt} C^0_{\nu a}$$  where $\ \ _a\ $ indexes the columns of the coefficient matrices, and the summation includes only the occupied spatial MOs.


In [22]:
F_0prime = np.dot(np.transpose(S_minhalf), np.dot(H_core, S_minhalf))

print("\n The initial Fock matrix in the orthogonal AO basis for the ", molecule, " test case is \n")
my_print(F_0prime)
print("\n",end="")


#F_0prime_eigval, F_0prime_eigvec = np.linalg.eig(F_0prime)    #we are'nt using this, as we want eigenvalues/eigenvectors in a definite(ascending/descending) order
def ascending_eigen(A):
  eigenValues, eigenVectors = np.linalg.eig(A)
  idx = np.argsort(eigenValues) #get the sorting-indexes(ascending) of the eigenvalues
  eigenValues = eigenValues[idx] #order the eigenvalues according to the sorting-indexes(ascending)
  eigenVectors = eigenVectors[:, idx] #order the eigenvectors according to the sorting-indexes(ascending)
  return (eigenValues, eigenVectors)

F_0prime_eigval, F_0prime_eigvec = ascending_eigen(F_0prime) # get eigenvalues/eigenvectors in a ascending order

epsilon_0 = np.dot(np.transpose(F_0prime_eigvec), np.dot(F_0prime, F_0prime_eigvec)) #we can also use matrix inverse instead of transpose 

C_0 = np.dot(S_minhalf, F_0prime_eigvec)

print("\n The initial MO coefficient matrix for the ", molecule, " test case is \n")
my_print(C_0)
print("\n",end="")


D_0 = np.zeros([nAO,nAO])

for mu in range(nAO):
  for nu in range(nAO):
    for a in range(int(N/2)):
      D_0[mu,nu] += C_0[mu,a]*C_0[nu,a]

print("\n The initial density matrix for the ", molecule, " test case is \n")
my_print(D_0)
print("\n",end="")


 The initial Fock matrix in the orthogonal AO basis for the  H₂O  test case is 

-32.2545866  -2.7914909  -0.0000000   0.0086110   0.0000000  -0.1812967  -0.1812967
 -2.7914909  -8.2368891   0.0000000  -0.2282926  -0.0000000  -0.3857987  -0.3857987
 -0.0000000   0.0000000  -7.5428890   0.0000000   0.0000000  -0.1132121   0.1132121
  0.0086110  -0.2282926   0.0000000  -7.4570295  -0.0000000  -0.1102196  -0.1102196
  0.0000000  -0.0000000   0.0000000  -0.0000000  -7.3471449  -0.0000000   0.0000000
 -0.1812967  -0.3857987  -0.1132121  -0.1102196  -0.0000000  -4.0329547  -0.0446466
 -0.1812967  -0.3857987   0.1132121  -0.1102196   0.0000000  -0.0446466  -4.0329547


 The initial MO coefficient matrix for the  H₂O  test case is 

  1.0015436  -0.2336245   0.0000000  -0.0856842  -0.0000000   0.0482226   0.0000000
 -0.0071893   1.0579388  -0.0000000   0.3601105   0.0000000  -0.4631213  -0.0000000
  0.0000000  -0.0000000  -1.0610702  -0.0000000   0.0000000   0.0000000  -0.2965071
 -0.0002671 

### Step 6: **Compute the Inital SCF Energy**

The SCF electronic energy may be computed using the density matrix as $$E^0_{elec} = \sum_{\mu\nu}^{AO} D^0_{\mu\nu} \hspace{1pt} \Big( H^{core}_{\mu\nu} + H^{core}_{\mu\nu} \Big)$$ 
$$$$
The total energy is the sum of the electronic energy and the nuclear repulsion energy $$E^0_{total} = E^0_{elec} + E_{nuc}$$ where $\ \ ^0 $ denotes the initial SCF energy.


In [23]:
E_elec = [] # this is a python list, not a numpy array, so that we can append the list
E_total = [] # this is a python list, not a numpy array, so that we can append the list

i=0  # iteration counter
E_elec.append(0)
E_total.append(0)

for mu in range(nAO):
  for nu in range(nAO):
    E_elec[i] += D_0[mu,nu] * ( H_core[mu,nu] + H_core[mu,nu] )

E_total[i] = E_elec[i] + E_nuc

print("\n The initial Hartree-Fock electronic energy for the ", molecule, " test case is {:18.12f} Hartrees. \n".format(E_elec[0]))


 The initial Hartree-Fock electronic energy for the  H₂O  test case is  -125.842077437699 Hartrees. 



## **The Self-Consistent Field Iteration**

### Step 7: **Compute the New Fock Matrix**

Start the SCF iterative procedure by building a new Fock matrix using the previous iteration's density as $$F^i_{\mu\nu} = H_{\mu\nu}^{core} + \sum_{\lambda\sigma}^{AO} \hspace{2pt} D_{\lambda\sigma}^{i-1} \hspace{2pt} \Big[ 2(\mu\nu|\lambda\sigma) - (\mu\lambda|\nu\sigma) \Big]$$ where the double-summation runs over all the AOs and $\ \ ^{i-1} $ denotes the density for the last iteration.

In [24]:
F = np.zeros([nAO,nAO])

def get_new_F(H_core,D,Eri):
  F = np.zeros([nAO,nAO])
  for mu in range(nAO):
    for nu in range(nAO):
      F[mu,nu] = H_core[mu,nu]
      for lam in range(nAO):
        for sig in range(nAO):
          F[mu,nu] += D[lam,sig]*( 2*Eri[mu,nu,lam,sig] - Eri[mu,lam,nu,sig] )
  return F

F = get_new_F(H_core,D_0,Eri)

print("\n The first-iteration Fock matrix in the AO basis for the ", molecule, " test case is \n")
my_print(F)
print("\n",end="")


 The first-iteration Fock matrix in the AO basis for the  H₂O  test case is 

-18.8132695  -4.8726875  -0.0000000  -0.0115290   0.0000000  -0.8067323  -0.8067323
 -4.8726875  -1.7909029  -0.0000000  -0.1808692   0.0000000  -0.5790557  -0.5790557
 -0.0000000  -0.0000000   0.1939644   0.0000000   0.0000000  -0.1708886   0.1708886
 -0.0115290  -0.1808692   0.0000000   0.2391247   0.0000000  -0.1828683  -0.1828683
  0.0000000   0.0000000   0.0000000   0.0000000   0.3091071   0.0000000   0.0000000
 -0.8067323  -0.5790557  -0.1708886  -0.1828683   0.0000000  -0.1450338  -0.1846675
 -0.8067323  -0.5790557   0.1708886  -0.1828683   0.0000000  -0.1846675  -0.1450338



### Step 8: **Build the New Density Matrix**

Form the new density matrix following the same procedure as in Step 5 above.

Orthogonalize $$\mathbf{F^\prime}^i = \mathbf{S^{-1/2}}^\mathsf{T} \hspace{2pt} \mathbf{F}^i \hspace{2pt} \mathbf{S^{-1/2}}$$

Diagonalize $$\mathbf{F^\prime}^i \hspace{2pt} \mathbf{C^\prime}^i=\mathbf{C^\prime}^i \hspace{2pt} \boldsymbol{\epsilon}^i$$

Back-transform $$\mathbf{C}^i=\mathbf{S^{-1/2}} \hspace{3pt} \mathbf{C^\prime}^i$$

Compute the density $$D_{\mu\nu}^i = \sum_a^{occ} \hspace{2pt} C^i_{\mu a} \hspace{1pt} C^i_{\nu a}$$ where $\ \ ^i $ denotes the current iteration density.

In [25]:
def get_new_D(F):
  F_prime = np.dot(np.transpose(S_minhalf), np.dot(F, S_minhalf)) #we can also use matrix inverse instead of transpose 
  
  F_prime_eigval, F_prime_eigvec = ascending_eigen(F_prime) # get eigenvalues/eigenvectors in a ascending order

  epsilon = np.dot(np.transpose(F_prime_eigvec), np.dot(F_prime, F_prime_eigvec)) #we can also use matrix inverse instead of transpose 

  C = np.dot(S_minhalf, F_prime_eigvec)


  D = np.zeros([nAO,nAO])

  for mu in range(nAO):
    for nu in range(nAO):
      for a in range(int(N/2)):
        D[mu,nu] += C[mu,a]*C[nu,a]
  
  return epsilon, C, D

epsilon, C, D = get_new_D(F)

### Step 9: **Compute the New SCF Energy**

Compute the new SCF energy as in step 6 above $$E_{elec}^i = \sum_{\mu\nu}^{occ} \hspace{2pt} D^i_{\mu\nu} \hspace{1pt} \Big( H^{core}_{\mu\nu} + F^i_{\mu\nu} \Big)$$

$$E_{total}^i = E_{elec}^i + E_{nuc}$$
 where $\ \ ^i $ denotes the SCF energy for the ith iteration.


In [26]:
def get_new_E_elec(D,H_core,F,i):
  E_elec.append(0)
  for mu in range(nAO):
    for nu in range(nAO):
      E_elec[i] += D[mu,nu]*(H_core[mu,nu]+F[mu,nu])
  return E_elec

def get_new_E_total(E_elec,E_nuc,i):
  E_total.append(0)
  E_total[i] = E_elec[i] + E_nuc
  return E_total

i=1
E_elec = get_new_E_elec(D,H_core,F,i)
E_total = get_new_E_total(E_elec,E_nuc,i)

### Step 10: **Test for Convergence**

Test both the energy and the density for convergence $$\Delta E^i = E_{elec}^i - E_{elec}^{i-1} < \delta_1$$

$${rms_D}^i = \left[ \sum_{\mu\nu} \Big( D_{\mu\nu}^i - D_{\mu\nu}^{i-1} \Big)^2 \right]^{1/2} < \delta_2$$

If the difference in consecutive SCF energy and the root-mean-squared difference in consecutive densities do not fall below the prescribed thresholds, return to Step 7 and continue from there.

In [27]:
Delta_E = [] # this is a python list, not a numpy array, so that we can append the list
Delta_E.append("") # 0th element is empty
rms_D = [] # this is a python list, not a numpy array, so that we can append the list
rms_D.append("") # 0th element is empty

def get_new_Delta_E(E_elec,i):
  Delta_E.append(0)
  Delta_E[i] = E_elec[i] - E_elec[i-1]
  return Delta_E

def get_new_rms_D(D_old,D,i):
  rms_D.append(0)
  for mu in range(nAO):
    for nu in range(nAO): 
      rms_D[i] += ( D[mu,nu] - D_old[mu,nu] )**2
  rms_D[i] = rms_D[i]**(1/2)
  return rms_D

i=1
D_old = D_0
Delta_E = get_new_Delta_E(E_elec,i)
rms_D = get_new_rms_D(D_old,D,i)



while rms_D[i]>1e-11:
  F = get_new_F(H_core,D,Eri)
  D_old = D
  epsilon, C, D = get_new_D(F)
  i += 1
  E_elec = get_new_E_elec(D,H_core,F,i)
  E_total = get_new_E_total(E_elec,E_nuc,i)
  Delta_E = get_new_Delta_E(E_elec,i)
  rms_D = get_new_rms_D(D_old,D,i)


print("\n The iteration-by-iteration electronic and total energies for the ", molecule, " test case are as follows \n")
df = pd.DataFrame(list(zip(E_elec,E_total,Delta_E,rms_D)), columns=['E(elec)','E(tot)','Delta(E)','RMS(D)'])
pd.options.display.float_format = '{:18.12f}'.format
pd.set_option('display.colheader_justify', 'center')
print(df)
#display(df)
print("\n",end="")


 The iteration-by-iteration electronic and total energies for the  H₂O  test case are as follows 

         E(elec)            E(tot)            Delta(E)            RMS(D)      
0   -125.842077437699  -117.839710375888                                      
1    -78.286583284740   -70.284216222929    47.555494152959     1.826673084479
2    -84.048316253435   -76.045949191625    -5.761732968696     0.479570364860
3    -82.716965960854   -74.714598899044     1.331350292581     0.086831688906
4    -82.987140757001   -74.984773695191    -0.270174796147     0.031026136359
5    -82.938133187513   -74.935766125703     0.049007569488     0.010799283179
6    -82.946271078699   -74.943904016889    -0.008137891186     0.005254826831
7    -82.944486784914   -74.942119723104     0.001784293785     0.002438579642
8    -82.944617252243   -74.942250190433    -0.000130467329     0.001177279531
9    -82.944503500703   -74.942136438892     0.000113751541     0.000564543180
10   -82.944478930625   -74.942

## **Additional Concepts**

### **The MO-Basis Fock Matrix**

At convergence, the canonical Hartree-Fock MOs are, by definition, eigenfunctions of the Fock operator, viz. $$F \chi_i = \epsilon_i \chi_i$$

If we multiply on the left by an arbitrary MO and integrate, we obtain $$F_{ij} \equiv \epsilon_i \delta_i = \langle \chi_j | F | \chi_i \rangle$$

In other words, the Fock matrix should be diagonal in the MO basis, with the orbital energies as its diagonal elements. We can demonstrate this explicitly using the AO-basis Fock matrix by first re-writing the above expression using the LCAO-MO coefficients $$F_{ij} = \sum_{\mu\nu} C_{\mu j} C_{\nu i} \langle \phi_\mu | F | \phi_\nu \rangle = \sum_{\mu\nu} C_{\mu j} C_{\nu i} F_{\mu\nu}$$

Use the above equation to transform the Fock matrix from the AO basis to the MO basis and demonstrate that it is indeed diagonal (to within the convergence limits of the SCF iterative procedure).

In [28]:
F_MO = np.zeros([nAO,nAO])

for i in range(nAO):
  for j in range(nAO):

    for mu in range(nAO):
      for nu in range(nAO):

        F_MO[i,j] += C[mu,j]*C[nu,i]*F[mu,nu]


print("\n Fock matrix in MO basis is \n")
my_print(F_MO) #actually same as epsilon
print("\n",end="")


 Fock matrix in MO basis is 

-20.2628916  -0.0000000  -0.0000000   0.0000000  -0.0000000   0.0000000   0.0000000
 -0.0000000  -1.2096974   0.0000000   0.0000000   0.0000000   0.0000000  -0.0000000
 -0.0000000   0.0000000  -0.5479646   0.0000000  -0.0000000   0.0000000  -0.0000000
  0.0000000   0.0000000   0.0000000  -0.4365272   0.0000000  -0.0000000   0.0000000
 -0.0000000   0.0000000  -0.0000000   0.0000000  -0.3875867  -0.0000000   0.0000000
  0.0000000   0.0000000   0.0000000  -0.0000000  -0.0000000   0.4776187  -0.0000000
  0.0000000   0.0000000   0.0000000  -0.0000000   0.0000000  -0.0000000   0.5881393



### **One-Electron Properties**

The calculation of one-electron properties requires density matrix and the relevant property integrals. The electronic contribution to the electric-dipole moment may be computed using, $$\vec{\mu}_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \vec{\mu}_{elec} | \phi_{\nu} \rangle $$ where the vector notation implies three sets of dipole-moment integrals -- one for each Cartesian component of the dipole operator. $$\mu^x_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^x_{elec} | \phi_{\nu} \rangle \quad \mu^y_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^y_{elec} | \phi_{\nu} \rangle \quad \mu^z_{elec} = 2\sum_{\mu\nu} D_{\mu\nu} \hspace{2pt} \langle \phi_{\mu} | \mu^z_{elec} | \phi_{\nu} \rangle $$ The factor 2 appearing above arises because the definition of the density used in this project differs from that used in Szabo & Ostlund.
$$$$
In order to compute the total dipole moment, we must include the nuclear contribution, which requires the atomic numbers $Z$ ( ≡ charge $q$) and Cartesian coordinates $\vec{r}$ of the nuclei $$\vec{\mu}_{nuc} = \sum_i \hspace{1pt} q_i \hspace{2pt} (\vec{r}_i - \vec{r}_{cm})$$ where $\vec{r}_{cm}$ is the Cartesian coordiate of centre-of-mass.
$$$$
Hence the total dipole moment $$\vec{\mu} = \vec{\mu}_{elec} + \vec{\mu}_{nuc}$$

In [29]:
Muxmunu = np.genfromtxt("mux.dat.txt")
Muymunu = np.genfromtxt("muy.dat.txt")
Muzmunu = np.genfromtxt("muz.dat.txt")

Mux = make_2Dmatrix(Muxmunu)
Muy = make_2Dmatrix(Muymunu)
Muz = make_2Dmatrix(Muzmunu)

def electronic_dipolemoment(D,Mux,Muy,Muz):
  Dipole_elec = np.zeros([3])
  
  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[0] += D[mu,nu]*Mux[mu,nu]
  Dipole_elec[0] *= 2

  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[1] += D[mu,nu]*Muy[mu,nu]
  Dipole_elec[1] *= 2

  for mu in range(nAO):
    for nu in range(nAO):
      Dipole_elec[2] += D[mu,nu]*Muz[mu,nu]
  Dipole_elec[2] *= 2

  return Dipole_elec

Dipole_elec = electronic_dipolemoment(D,Mux,Muy,Muz)


geom3 = np.genfromtxt("geom.dat.txt", skip_header=1)
geom3=add_mass_column(geom3) # now the geom array has a 4th column, for the mass of the atoms

cm = center_of_mass(geom3)

def nuclear_dipolemoment(geom):
  Dipole_nuc = np.zeros([3])
  for i in range(3): #for each coordinate
    for A in range(len(geom)): #for each atom
      Dipole_nuc[i] += geom[A,0]*(geom[A,i+1]-cm[i])
  return Dipole_nuc

Dipole_nuc = nuclear_dipolemoment(geom3)


Dipole = np.add(Dipole_elec, Dipole_nuc)


print("\n Dipole moment \n")
print("\u03bc\u02e3 =  {:16.12f} \n\u03bc\u02b8 =  {:16.12f} \n\u03bc\u1dbb =  {:16.12f} \n".format(Dipole[0],Dipole[1],Dipole[2]))


 Dipole moment 

μˣ =    0.000000000000 
μʸ =    0.603521296526 
μᶻ =   -0.000000000000 



### **Population Analysis/Atomic Charges**

A Mulliken population analysis requires the overlap integrals and the electron density, in addition to information about the number of basis functions centered on each atom. The charge on atom A may be computed as $$q_A = Z_A - 2 \sum_{\mu\in A} (\mathbf{DS})_{\mu\mu}$$ where the summation is limited to only those basis functions centered on atom A.

In [30]:
def sum_arr_upto(X,n): # X is the 1D array, n is upto how many elements we want the sum
  sum = 0
  for i in range(n):
    sum += X[i]
  return sum


q = np.zeros([len(geom3)])

print("\n Atomic charges \n")
for A in range(len(geom3)):
  thesum = 0
  for mu in range(sum_arr_upto(n_basis,A),sum_arr_upto(n_basis,A+1)):
    thesum += np.dot(D,S)[mu,mu]
  q[A] = geom3[A,0] - 2*thesum
  print("Charge on atom {:1.0f} =   {:16.12f}".format(A,q[A]))
print("\n",end="")


 Atomic charges 

Charge on atom 0 =    -0.253146052405
Charge on atom 1 =     0.126573026202
Charge on atom 2 =     0.126573026202



# Project 4: **The second-order Moller-Plesset perturbation (MP2) energy**

Unlike the Hartree-Fock energy, correlation energies like the MP2 energy are usually expressed in terms of MO-basis quantities (integrals, MO energies).

### Step 1: **Read the Two-Electron Integrals**


Same as Step 3 of Project 3

### Step 2: **Obtain the MO Coefficients and MO Energies**


The MO Coefficients values $C_{\mu p}$ and MO energies $\epsilon_p$ computed in the Hartree-Fock program of Project 3.

### Step 3: **Transform the Two-Electron Integrals to the MO Basis**


This is the most expensive part of the calculation.
$$$$

**Noddy Algorithm**

The most straightforward expression of the AO/MO integral transformation is $$(pq|rs) = \sum_\mu \sum_\nu \sum_\lambda \sum_\sigma C_{\mu p} \hspace{1pt} C_{\nu q} \hspace{1pt} (\mu\nu|\lambda\sigma) \hspace{1pt} C_{\lambda r} \hspace{1pt} C_{\sigma s}$$ This approach is easy to implement, but is expensive due to its $\mathcal{O(n^8)}$ computational order.


In [31]:
Eri_MO = np.zeros([nAO,nAO,nAO,nAO])

for p in range(nAO):
  for q in range(nAO):
    for r in range(nAO):
      for s in range(nAO):

        for mu in range(nAO):
          for nu in range(nAO):
            for lam in range(nAO):
              for sig in range(nAO):

                Eri_MO[p,q,r,s] += C[mu,p]*C[nu,q]*C[lam,r]*C[sig,s]*Eri[mu,nu,lam,sig]


### Step 4: **Compute the MP2 Energy**

$$E_{MP2} = \sum_{ij} \sum_{ab} \frac{(ia|jb) \hspace{1pt} \Big[ 2(ia|jb) - (ib|ja) \Big]}{\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b}$$ 
where $i$ and $j$ denote doubly-occupied orbitals and $a$ and $b$ unoccupied orbitals, and the denominator involves the MO energies.

In [32]:
Emp2 = 0

for i in range(int(N/2)):
  for j in range(int(N/2)):

    for a in range(int(N/2),nAO):
      for b in range(int(N/2),nAO):

        Emp2 += Eri_MO[i,a,j,b] * (2*Eri_MO[i,a,j,b]-Eri_MO[i,b,j,a]) / (epsilon[i,i]+epsilon[j,j]-epsilon[a,a]-epsilon[b,b])


Escf = E_total[-1]
Etot = Escf + Emp2

print("\n Escf =  {:18.12f}".format(Escf))
print(" Emp2 =  {:18.12f}".format(Emp2))
print("\u23af"*30)
print(" Etot =  {:18.12f} \n".format(Etot))


 Escf =    -74.942079928192
 Emp2 =     -0.049149636120
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
 Etot =    -74.991229564312 

